In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

In [2]:
#create path
path = r'/Users/rachelallen/CareerFoundry Achievement 4/08-2022 Instacart Basket Analysis/02 Data'

In [4]:
#import as ords_prods_merge
ords_prods_merge = pd.read_pickle(os.path.join(path, 'Prepared Data', 'orders_products_merged_updated.pkl'))

In [5]:
#create subset of first million rows
df = ords_prods_merge[:1000000]

In [7]:
#check shape
df.shape

(1000000, 20)

In [8]:
df.head()

,order_id,user_id,order_number,order_day_of_week,order_hour,days_since_prior_order,flagged,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,Exists,price_range_loc,busiest_day,busiest_days,busiest_period_of_day
0,2539329.0,1.0,1.0,2.0,8.0,NaN,No Previous Order,196,1.0,0.0,both,Soda,77.0,7.0,9.0,both,NaN,Regularly busy,Regularly busy,Average orders
1,2398795.0,1.0,2.0,3.0,7.0,15.0,Previous Order,196,1.0,1.0,both,Soda,77.0,7.0,9.0,both,NaN,Regularly busy,Least busy days,Average orders
2,473747.0,1.0,3.0,3.0,12.0,21.0,Previous Order,196,1.0,1.0,both,Soda,77.0,7.0,9.0,both,NaN,Regularly busy,Least busy days,Average orders
3,2254736.0,1.0,4.0,4.0,7.0,29.0,Previous Order,196,1.0,1.0,both,Soda,77.0,7.0,9.0,both,NaN,Least busy,Least busy days,Average orders
4,431534.0,1.0,5.0,4.0,15.0,28.0,Previous Order,196,1.0,1.0,both,Soda,77.0,7.0,9.0,both,NaN,Least busy,Least busy days,Average orders


In [9]:
#try groupby()
df.groupby('product_name')

In [11]:
#group by and then agg lets you see highest selling by department for example
df.groupby('department_id').agg({'order_number': ['mean']})


,order_number
,mean
department_id,
4.0,18.825780
7.0,17.472355
13.0,17.993423
14.0,19.246334
16.0,19.463012
17.0,11.294069
19.0,19.305237
20.0,17.599636


In [12]:
df.groupby('department_id')['order_number'].mean()


department_id
4.0     18.825780
7.0     17.472355
13.0    17.993423
14.0    19.246334
16.0    19.463012
17.0    11.294069
19.0    19.305237
20.0    17.599636
Name: order_number, dtype: float64

In [13]:
#apply multiple aggregates
df.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']})

order_number           
                      mean  min   max
department_id                        
4.0              18.825780  1.0  99.0
7.0              17.472355  1.0  99.0
13.0             17.993423  1.0  99.0
14.0             19.246334  1.0  99.0
16.0             19.463012  1.0  99.0
17.0             11.294069  1.0  98.0
19.0             19.305237  1.0  99.0
20.0             17.599636  1.0  99.0

In [14]:
#try the code
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [15]:
ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour,days_since_prior_order,flagged,product_id,add_to_cart_order,reordered,...,product_name,aisle_id,department_id,prices,Exists,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329.0,1.0,1.0,2.0,8.0,NaN,No Previous Order,196,1.0,0.0,...,Soda,77.0,7.0,9.0,both,NaN,Regularly busy,Regularly busy,Average orders,10.0
1,2398795.0,1.0,2.0,3.0,7.0,15.0,Previous Order,196,1.0,1.0,...,Soda,77.0,7.0,9.0,both,NaN,Regularly busy,Least busy days,Average orders,10.0
2,473747.0,1.0,3.0,3.0,12.0,21.0,Previous Order,196,1.0,1.0,...,Soda,77.0,7.0,9.0,both,NaN,Regularly busy,Least busy days,Average orders,10.0
3,2254736.0,1.0,4.0,4.0,7.0,29.0,Previous Order,196,1.0,1.0,...,Soda,77.0,7.0,9.0,both,NaN,Least busy,Least busy days,Average orders,10.0
4,431534.0,1.0,5.0,4.0,15.0,28.0,Previous Order,196,1.0,1.0,...,Soda,77.0,7.0,9.0,both,NaN,Least busy,Least busy days,Average orders,10.0


In [16]:
ords_prods_merge['max_order'].value_counts(dropna = False)

99.0    1172313
8.0      812631
6.0      812072
9.0      810842
7.0      804745
         ...   
90.0      48399
97.0      44969
98.0      44604
96.0      40469
NaN          11
Name: max_order, Length: 98, dtype: int64

In [17]:
#create flag with .loc()
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [18]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [19]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [21]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15891507
Loyal customer      10294027
New customer         6249525
NaN                       11
Name: loyalty_flag, dtype: int64

In [22]:
ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head(60)

,user_id,loyalty_flag,order_number
0,1.0,New customer,1.0
1,1.0,New customer,2.0
2,1.0,New customer,3.0
3,1.0,New customer,4.0
4,1.0,New customer,5.0
5,1.0,New customer,6.0
6,1.0,New customer,7.0
7,1.0,New customer,8.0
8,1.0,New customer,9.0
9,1.0,New customer,10.0
